In [113]:
import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [132]:
class FullyConnectedLayer():
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        np.random.seed(42)
        # Initialize weights and biases with Xavier initialization
        self.w = np.random.normal(loc=0, scale=1/np.sqrt(self.input_size), size=(self.input_size, self.output_size))
        self.b = np.random.normal(loc=0, scale=1/np.sqrt(self.input_size), size=(self.output_size))
    def __call__(self, x):
        y = np.matmul(x, self.w) + self.b
        return y
    def backward(self, grad, x):
        self.grad_w = np.outer(x, grad)
        self.grad_b = grad
        return self.grad_w, self.grad_b
    def update(self, learning_rate=0.01):
        self.w -= learning_rate * self.grad_w 
        self.b -= learning_rate * self.grad_b

class Sigmoid():
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))
    def backward(self, grad, x):
        if len(grad.shape) > 1:
            grad = np.sum(grad, axis=1)
        return grad * (np.exp(-x)) / (1 + np.exp(-x))**2

class MSE():
    def __call__(self, y_pred, y_gt):
        return (y_gt - y_pred)**2
    def backward(self, y_pred, y_gt):
        return 2 * (y_pred - y_gt)

class Layer():
    def __init__(self, fcl):
        self.fcl = fcl

class MLP():
    def __init__(self, layers_sizes):
        self.layers_sizes = layers_sizes
        self.sigmoid = Sigmoid()
        self.mse = MSE()
        self.layers = []
        for i in range(len(layers_sizes) - 1):
            fcl = FullyConnectedLayer(layers_sizes[i], layers_sizes[i+1])
            self.layers.append(Layer(fcl))
    def __call__(self, x):
        for layer in self.layers:
            layer.input = x
            layer.fcl_out = layer.fcl(x)
            layer.sigmoid_out = self.sigmoid(layer.fcl_out)
            x = layer.sigmoid_out
        return x
    def backward(self, y_gt):
        grad = self.mse.backward(self.layers[-1].sigmoid_out, y_gt)
        for layer in reversed(self.layers):
            grad_sigmoid = self.sigmoid.backward(grad, layer.fcl_out)
            grad, _ = layer.fcl.backward(grad_sigmoid, layer.input)
    def update(self, learning_rate=0.01):
        for layer in self.layers:
            layer.fcl.update(learning_rate)

In [133]:
x = np.array([1, 1, 1])
y_gt = np.array([1, 1])
input_size = len(x)
output_size = len(y_gt)

mlp = MLP([input_size, 10, 10, 10, 10, output_size])

In [134]:
learning_rate = 0.01
n_epochs = 100000

for i in tqdm.tqdm(range(n_epochs)):
    y = mlp(x)
    loss = mlp.mse(y, y_gt)
    mlp.backward(y_gt)
    mlp.update()
print(y)

100%|██████████| 100000/100000 [00:14<00:00, 7072.09it/s]

[0.99424789 0.99433034]


In [200]:
x = np.array([1, 1, 1])
y_gt = np.array([1, 1])

input_size = len(x)
output_size = len(y_gt)
print(f"input shape: {x.shape}, output shape: {y.shape}")
print(f"input: {x}")
print(f"output: {y}")
print("----------------------------------------------------------------")

fcl = FullyConnectedLayer(input_size, output_size)
print(f"Weights: {fcl.w}")
print(f"Biases: {fcl.b}")
print("----------------------------------------------------------------")

input shape: (3,), output shape: (2,)
input: [1 1 1]
output: [1 1]
----------------------------------------------------------------
Weights: [[ 0.28677805 -0.07982693]
 [ 0.37394315  0.8793217 ]
 [-0.13518851 -0.13517904]]
Biases: [0.91175894 0.44307865]
----------------------------------------------------------------


In [201]:
learning_rate = 0.001
n_epochs = 100000

for i in range(1, n_epochs):
    # Forward
    out_fcl = fcl(x)

    sigmoid = Sigmoid()
    out_sigmoid = sigmoid(out_fcl)
    # print(f"Out sigmoid: {out_sigmoid}")

    mse = MSE()
    out_mse = mse(out_sigmoid, y_gt)
    # print("--------------------------------")

    # Backward
    grad_loss = mse.backward(out_sigmoid, y_gt)
    # print(f"grad_loss: {grad_loss}")
    grad_activation = sigmoid.backward(grad_loss, out_fcl)
    # print(f"grad_activation: {grad_activation}")
    fcl.backward(grad_activation, x)
    fcl.update(learning_rate=learning_rate)
    # print(f"Weights: {fcl.w}")
    # print(f"Biases: {fcl.b}")

print(f"Out sigmoid: {out_sigmoid}")
print(f"out_mse: {out_mse}")

Out sigmoid: [0.97401082 0.9738612 ]
out_mse: [0.00067544 0.00068324]
